<a href="https://colab.research.google.com/github/SY-256/anomaly_detection/blob/main/notebook/chapter6_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 多変数のホテリング理論による異常検知

多変数のホテリング理論による異常検知
- A.変数選択
- B.モデルの学習
- C.推論

## A.変数選択

In [ ]:
# ヒストグラムによる正常と異常の分離性の確認
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("https://raw.githubusercontent.com/ghmagazine/python_anomaly_detection_book/refs/heads/main/notebooks/datasets/ch2_dataset_train.csv")
# "label"列を削除（数値型の列のみとする）
df_val = df.drop("label", axis=1)
# 正常データのみ取り出す
df_normal_val = df[df["label"] == "normal"].drop("label", axis=1)
# 異常データのみ取り出す
df_anomaly_val = df[df["label"] == "anomaly"].drop("label", axis=1)
fig, ax = plt.subplots(nrows=1, ncols=5, figsize=(20, 4))

# すべての変数をループで走査
for i, colname in enumerate(df_normal_val.columns):
    # 正常データのヒストグラム描画
    sns.histplot(data=df_normal_val[colname], bins="sturges",
                 color="#bbbbbb", ax=ax[i],
                 stat="density", label="normal")
    # 異常データのヒストグラム描画
    sns.histplot(data=df_anomaly_val[colname], bins="sturges",
                 color="#111111", ax=ax[i],
                 stat="density", label="anomaly")

    # 凡例を追加
    ax[i].legend()
    # 変数名を図のタイトルとして追加
    ax[i].set_title(colname, fontsize=14)

plt.tight_layout()
plt.show()

変数`temp1`、`temp2`、`temp3`において正常と異常の分布がある程度分離しているもの対し、`temp4`と`temp5`は両者の分布に目に見える差がない。よって、本データの異常の検知に寄与する可能性が低く、モデルの入力する必要性は低いと判断できる。

In [ ]:
# 散布図（pairplot）による相関の確認
sns.pairplot(
    data=df,
    vars=["temp1", "temp2", "temp3"],
    hue="label",
    palette=["#999999", "#111111"]
)
plt.show()

ホテリング理論は変数間の相関に比較的強いアルゴリズムであるので、弱相関程度であれば変数を削除しなくても良い。（今回は便宜上モデルの判定を可視化し易くするために、入力変数は`temp1`と`temp2`の2変数に絞る。

## B.モデルの学習

学習フェーズでは、最尤推定による多次元正規分布パラメータの推定、および異常度の閾値算出を実施

In [ ]:
# 多変数のホテリング理論による異常検知の実装例（学習）
# N >> Mが成り立つ場合の学習の実装（カイ二乗分布）
import pandas as pd
import numpy as np
from scipy import stats

##### 学習データの読み込みと前処理 #####
df = pd.read_csv("https://raw.githubusercontent.com/ghmagazine/python_anomaly_detection_book/refs/heads/main/notebooks/datasets/ch2_dataset_train.csv")
# "temp2"と"temp1"に欠損があるデータを削除
df_dropna = df.dropna(subset=["temp1", "temp2"], axis=0)
# 正常データのみを抽出
df_normal = df_dropna[df_dropna["label"] == "normal"]
# "temp2", "temp1"列のみ取り出してNumpyのndarray化し、学習データとする
X_train = df_normal[["temp2", "temp1"]].to_numpy()

###### 学習ステップ1. 正常のモデルを作成 #####
mu = np.mean(X_train, axis=0) # 標本平均ベクトルμを算出
# 標本分散共分散行列Σを算出（転置と自由度ddofに注意）
Sigma = np.cov(X_train.T, ddof=0)

##### 学習ステップ2. 異常を表す指標（異常度）を定義する #####
# 式のみの定義

#### 学習ステップ3. 異常度に閾値を設ける #####
TARGET_FP_RATE = 0.0027 # ターゲットとする誤報率（正規分布の3σ相当=0.0027）
n_features = X_train.shape[1] # 変数の数M
# 自由度（M）のカイ二乗分布の累積分布関数の逆関数から閾値を算出
a_th = stats.chi2.ppf(1-TARGET_FP_RATE, df=n_features)

##### 学習で求めたパラメータをすべて表示 #####
print(f"mu={mu}")
print(f"Sigma={Sigma}")
print(f"a_th={a_th}")

In [ ]:
# N >> Mが成り立たない場合の学習の実装（F分布 自由度(M, N - M)）
##### 学習ステップ1. 正常のモデルを作成する #####
mu = np.mean(X_train, axis=0) # 標本平均ベクトルμを算出
# 標本分散共分散行列Σを算出
Sigma = np.cov(X_train.T, ddof=0)

##### 学習ステップ2. 異常を表す指標（異常度）を算出 #####
# 式を定義するのみ

##### 学習ステップ3. 異常度に閾値を設ける #####
TARGET_FP_RATE = 0.0027 # ターゲットとする誤報率（正規分布の3σ相当=0.0027）
sample_size = len(X_train) # サンプルサイズN
n_features = X_train.shape[1] # 変数の数M
# 自由度（M, N - M）のF分布の累積分布関数の逆関数から閾値を算出
a_th = (sample_size+1)*n_features/(sample_size-n_features) \
* stats.f.ppf(1-TARGET_FP_RATE, dfn=n_features, dfd=sample_size-n_features)

##### 学習で求めたパラメータを表示 #####
print(f"mu={mu}")
print(f"Sigma={Sigma}")
print(f"a_th={a_th}")

カイ二乗分布を使用した場合と比較して、異常度の閾値がやや大きくなっており、誤報率を減らす方向（見逃し寄り）に閾値が設定されてることがわかる（右側より）